<a href="https://colab.research.google.com/github/Vishwas2629/Dissertation/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re

In [ ]:
dataset = pd.read_csv('IMDB_Dataset.csv', encoding = 'utf-8')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
import pickle


In [ ]:
imdb_data_len = len(dataset.iloc[:, 0])
print(imdb_data_len)

50000


In [ ]:
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

ps = PorterStemmer()


# new list with cleaned data
corpus = []
for i in range(imdb_data_len): # this can also be written as imdb_data['review']
  review = BeautifulSoup(dataset['review'][i], "html.parser").get_text()
  review = re.sub('\[[^]]*\]',' ',review)
  review = re.sub('[^a-zA-Z]',' ',review)
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)

  corpus.append(review)
    

In [ ]:
print(corpus[3])

basic famili littl boy jake think zombi closet parent fight time movi slower soap opera suddenli jake decid becom rambo kill zombi ok first go make film must decid thriller drama drama movi watchabl parent divorc argu like real life jake closet total ruin film expect see boogeyman similar movi instead watch drama meaningless thriller spot well play parent descent dialog shot jake ignor


In [ ]:
#Creating bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [ ]:
print(X[3])

[0 0 0 ... 2 0 0]


In [ ]:
y = dataset['sentiment']
y = np.array(list(map(lambda x: 1 if x == "positive" else 0, y)))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def metrics(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Classification Report:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix:\n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Classification Report:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix:\n {confusion_matrix(y_test, pred)}\n")

## Decision Tree 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier1 = DecisionTreeClassifier(criterion= "gini", random_state= 0, max_depth = 5)
classifier1.fit(X_train, y_train)

In [ ]:
y_pred = classifier1.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
metrics(classifier1, X_train, y_train, X_test, y_test, train=True)
metrics(classifier1, X_train, y_train, X_test, y_test, train=False)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier1, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("SD: {:.2f} %".format(accuracies.std()*100))


# XG Boost Classifier

In [ ]:
from xgboost import XGBClassifier
classifier3 = XGBClassifier(max_depth = 5, n_estimators = 10, random_state= 0)
classifier3.fit(X_train, y_train)

In [ ]:
y_pred = classifier3.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
metrics(classifier3, X_train, y_train, X_test, y_test, train=True)
metrics(classifier3, X_train, y_train, X_test, y_test, train=False)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier3, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("SD: {:.2f} %".format(accuracies.std()*100))


## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier5 = GaussianNB()
classifier5.fit(X_train,y_train)

In [ ]:
y_pred = classifier5.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
metrics(classifier5, X_train, y_train, X_test, y_test, train=True)
metrics(classifier5, X_train, y_train, X_test, y_test, train=False)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier5, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("SD: {:.2f} %".format(accuracies.std()*100))


# SVM

In [ ]:
from sklearn.linear_model import SGDClassifier
classifierSVM = SGDClassifier(alpha = 0.1, random_state = 0)
classifierSVM.fit(X_train, y_train)

SGDClassifier(alpha=0.1, random_state=0)

In [ ]:
y_pred = classifierSVM.predict(X_test)     
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [0 0]
 [1 0]
 ...
 [1 1]
 [0 1]
 [0 0]]


In [ ]:
metrics(classifierSVM, X_train, y_train, X_test, y_test, train=True)
metrics(classifierSVM, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 86.18%
_______________________________________________
Classification Report:
                      0             1  accuracy     macro avg  weighted avg
precision      0.879787      0.845560  0.861825      0.862674      0.862644
recall         0.837616      0.885950  0.861825      0.861783      0.861825
f1-score       0.858184      0.865284  0.861825      0.861734      0.861740
support    19965.000000  20035.000000  0.861825  40000.000000  40000.000000
_______________________________________________
Confusion Matrix:
 [[16723  3242]
 [ 2285 17750]]

Test Result:
Accuracy Score: 85.13%
_______________________________________________
Classification Report:
                     0            1  accuracy     macro avg  weighted avg
precision     0.867745     0.835974    0.8513      0.851859      0.851970
recall        0.831380     0.871501    0.8513      0.851440      0.851300
f1-score      0.849173     0.853368    0.8513      0.851270      0.851256
support   

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifierSVM, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("SD: {:.2f} %".format(accuracies.std()*100))


Accuracy: 85.55 %
SD: 0.47 %


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifierRM = RandomForestClassifier(n_estimators= 5, max_depth = 5,criterion= 'entropy', random_state = 0 )
classifierRM.fit(X_train, y_train)

In [ ]:
y_pred = classifierRM.predict(X_test)     
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
metrics(classifierRM, X_train, y_train, X_test, y_test, train=True)
metrics(classifierRM, X_train, y_train, X_test, y_test, train=False)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifierRM, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("SD: {:.2f} %".format(accuracies.std()*100))
